# Imports

In [1]:
import rasterio
import geopandas as gpd
from rasterio.features import rasterize
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

import xarray
import glob
from PIL import Image
from rioxarray.merge import merge_arrays
import rioxarray
import matplotlib.pyplot as plt

In [2]:
from zipfile import ZipFile
from io import BytesIO

In [3]:
import os
import requests
from datetime import datetime

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
def download_tif(url, dir_path):
    response = requests.get(url, verify=False)
    if response.status_code == 200:
        with ZipFile(BytesIO(response.content)) as zip_ref:
            # Loop through each file in the zip
            for zip_info in zip_ref.infolist():
                # Extract only files, ignore directories
                if not zip_info.is_dir():
                    # Use the file name only, ignoring any directory paths in the zip file
                    zip_info.filename = os.path.basename(zip_info.filename)
                    # Extract the file with the new path
                    zip_ref.extract(zip_info.filename, path=dir_path) 
    else:
        print(f"{datetime.now()} {url} failed")

In [6]:
current_dir = os.getcwd()
DSM_dir_path = current_dir + r"\DSM"
DTM_dir_path = current_dir + r"\DTM"
RGB_NDSM_dir_path = current_dir + r"\RGB_NDSM"

In [7]:
def create_dir(path):
    # Check if the directory exists
    if not os.path.exists(path):
        # Create the directory if it does not exist
        os.makedirs(path)
        print(f"Directory '{path}' was created.")
    else:
        print(f"Directory '{path}' already exists.")

In [8]:
create_dir(DSM_dir_path)
create_dir(DTM_dir_path)
create_dir(RGB_NDSM_dir_path)

Directory 'C:\Users\Niels\PycharmProjects\SSML-group-project\DSM' already exists.
Directory 'C:\Users\Niels\PycharmProjects\SSML-group-project\DTM' already exists.
Directory 'C:\Users\Niels\PycharmProjects\SSML-group-project\RGB_NDSM' already exists.


In [9]:
DSM_files = ["https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_31GN2.zip",
                "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_31HN1.zip",
                "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_31HN2.zip",
                "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_32CN1.zip",
                "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_31GZ2.zip",
                "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_31HZ1.zip",
                "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_31HZ2.zip",
                "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_32CZ1.zip",
                "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_38EN2.zip",
                "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_38FN1.zip",
                "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_38FN2.zip",
                "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m/R_39AN1.zip"]

DTM_files = ["https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_31GN2.zip",
             "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_31HN1.zip",
             "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_31HN2.zip",
             "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_32CN1.zip",
             "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_31GZ2.zip",
             "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_31HZ1.zip",
             "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_31HZ2.zip",
             "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_32CZ1.zip",
             "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_38EN2.zip",
             "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_38FN1.zip",
             "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_38FN2.zip",
             "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m/M_39AN1.zip"]

In [10]:
for DSM_file in tqdm(DSM_files):
    download_tif(DSM_file, DSM_dir_path)

In [11]:
for DTM_file in tqdm(DTM_files):
    download_tif(DTM_file, DTM_dir_path)

### Create NDSM

In [12]:
# Use glob to find all TIFF files in the directory
tif_files = glob.glob(f"{DTM_dir_path}/*.tif")

# Load each TIFF file
DTMs = []
for file_path in tif_files:
    DTM = rioxarray.open_rasterio(file_path)
    DTMs.append(DTM)

In [13]:
# Use glob to find all TIFF files in the directory
tif_files = glob.glob(f"{DSM_dir_path}/*.tif")

# Load each TIFF file
DSMs = []
for file_path in tif_files:
    DSM = rioxarray.open_rasterio(file_path)
    DSMs.append(DSM)

In [14]:
# Merge all the raster datasets into one big raster
merged_DTM = merge_arrays(DTMs)
merged_DSM = merge_arrays(DSMs)

In [15]:
no_data_value = 30000

merged_DTM = merged_DTM.where(merged_DTM < no_data_value, np.nan)
merged_DSM = merged_DSM.where(merged_DSM < no_data_value, np.nan)

In [16]:
NDSM = merged_DSM - merged_DTM

In [17]:
del merged_DSM
del merged_DTM

### Write files with 4 bands

In [ ]:
rgb_files = glob.glob("nl_8cm/*.tif")

for rgb_file in tqdm(rgb_files):
    # Load rgb file
    rgb = rioxarray.open_rasterio(rgb_file)
    bounds = rgb.rio.bounds()
    # Slice rgb file outline from NDSM file
    sub_NDSM = NDSM.rio.clip_box(minx=bounds[0], miny=bounds[1], maxx=bounds[2], maxy=bounds[3])
    # Upsample to same resolution as RGB images
    sub_NDSM = sub_NDSM.interp(y=rgb.y, x=rgb.x, method='nearest')
    # Merge RGB with NDSM
    combined = xarray.concat([rgb, sub_NDSM], dim='band')
    # Write R,G,B,NDSM file
    filename = rgb_file.split("\\")[-1]  # Corrected: Escape the backslash or use a raw string if needed
    combined.rio.to_raster(f"RGB_NDSM/{filename}")

 23%|█████████████████████████████████▏                                                                                                                | 15/66 [04:35<15:24, 18.12s/it]